In [2]:
%reload_ext autoreload
%autoreload 2
import sys, os
print(sys.executable)
module_path = os.path.abspath(os.path.join('..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)
# Verify kernal path
import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


True

In [3]:
import src.config as config
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
2024-08-06 20:17:09,818 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-08-06 20:17:09,818 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Feature view is defined to read data from feature store, define how you want to fetch data from potentially many different feature groups and merge it to get the final dataset. 
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print("Feaeture view already existed. Skip creation")

feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feaeture view already existed. Skip creation


In [1]:
# feature-view-obj.func(): gets training data-frame from hopsworks
ts_prices, _ = feature_view.training_data(
    description="Time-series daily stock prices"
)

NameError: name 'feature_view' is not defined

In [ ]:
# sort the data fetched from feature-store and make sure it is sorted, the last row should be the current date because we are getting most recetn data with feature pipeline
ts_prices.sort_values(by=["datetime"], inplace=True)
ts_prices

In [ ]:
from src.data import transform_ts_data_into_features_target

n_previous_days = 12
step_size = 1
features, targets = transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size)
print(f'Features shape: {features.shape}')  # Should be (number_of_examples, n_previous_days, n_features)
print(f'Targets shape: {targets.shape}')    # Should be (number_of_examples,)


In [ ]:
from src.data import train_test_split_tabular

# combine features/targets to create tabular data
features_and_target = pd.DataFrame(features.copy())
features_and_target["target_close_price_next_day"] = targets   # target column name
print(f"{features_and_target.shape=}")
features_and_target

In [ ]:
from sklearn.model_selection import train_test_split
from src.data import train_test_split_tabular

X_train, X_test, y_train, y_test = train_test_split_tabular(features_and_target)
print(f"X_train: {X_train.shape}") # (examples-in-train, num-features)
print(f"y_train: {y_train.shape}") # (examples-in-train,) 1D beceause only one target
print(f"X_test: {X_test.shape}")   # (examples-in-test, num-features)
print(f"y_test: {y_test.shape}")

In [ ]:
# CREATE MODEL
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)

# Train the model
lgb_model.fit(features, targets)

# Predict on the test set
y_pred = lgb_model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

In [44]:
# SAVE MODEL AS PICKLE LOCALLY
import joblib

model_path = f'../models/lgb_model.pkl'
joblib.dump(lgb_model, model_path)


['../models/lgb_model.pkl']

In [45]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# before saving model in hopsworks need to define its schema
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)


In [46]:
# Connect to the Hopsworks model registry
model_registry = project.get_model_registry()

# Register the model
model = model_registry.sklearn.create_model(
    name="lightgbm_model_stock_prices",
    metrics={"MSE": mse},
    description="LightGBM model for predicting stock prices",
    input_example=X_train.sample(),
    model_schema=model_schema
)

# Upload the model file
model.save(model_path)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/274456 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3925 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/14679 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/835736/models/lightgbm_model_stock_prices/2


Model(name: 'lightgbm_model_stock_prices', version: 2)